In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
'''sys.path.append('/home/wangs/rs/lwm/lib')'''
import ff
import pandas as pd
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib as mpl
mpl.rc("font", family='Droid Sans Fallback', weight="bold")
from datetime import datetime
import statsmodels.api as sm
from scipy import stats
from scipy.stats import poisson

In [98]:
import psutil
import os

print(u'当前进程的内存使用：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 ** 3) )

info = psutil.virtual_memory()
print(u'电脑总内存：%.4f GB' % (info.total / 1024 ** 3) )
print(u'当前使用的总内存占比：',info.percent)
print(u'cpu个数：',psutil.cpu_count())

当前进程的内存使用：7.7660 GB
电脑总内存：1007.4503 GB
当前使用的总内存占比： 62.9
cpu个数： 256


# 基础因子

## 剔除了开盘和收盘 10 分钟数据（没做这步）

## 一阶自相关系数
升序排列

In [3]:
import os
folder_path = '/home/wangs/data/ba/'
names = []
for file in os.listdir(folder_path):
    if file.endswith('.h5'):
        file_name = os.path.splitext(file)[0] 
        names.append(file_name)

In [4]:
def get_data(para):
    name,fre = para
    data =  pd.DataFrame()
    ini_data = ff.read_binance(name).loc[:'20240216 235900']
    ini_data.index = pd.to_datetime(ini_data.index)
    v_day = ini_data.v.resample('D').sum()
    v_fre = ini_data.v.resample(f'{fre}T').sum()
    data.loc[:,'v_fra'] = v_fre/v_day[v_fre.index.date].values
    data.loc[:,'ret'] = ini_data.ret.resample(f'{fre}T').apply(lambda x: (x+1).prod()-1)
    return data

In [5]:
def get_data_simple(para):
    name,fre = para
    ini_data = ff.read_binance(name).ret.loc[:'20240216 235900']
    ini_data.index = pd.to_datetime(ini_data.index)
    if fre == 1:
        ret = ini_data.resample('1T').asfreq()
        ret.name = name
    else:
        ret = ini_data.resample(f'{fre}T').apply(lambda x: (x+1).prod()-1)
        ret.name = name
    return ret

In [35]:
def get_data_simple2(para):
    name,fre = para
    ini_data = ff.read_binance(name).c.loc[:'20240216 235900']
    ini_data.index = pd.to_datetime(ini_data.index)
    if fre == 1:
        close = ini_data.resample('1T').asfreq()
        close.name = name
    else:
        close = ini_data.resample(f'{fre}T').last()
        close.name = name
    return close

In [7]:
def get_foc(para):
    data = get_data(para)
    rtn_foc = data.ret.resample('D').apply(lambda x: x.autocorr())
    vol_foc = data.v_fra.resample('D').apply(lambda x: x.autocorr())
    foc_Comb = (rtn_foc+vol_foc)*0.5
    rtn_foc.name = para[0]
    vol_foc.name = para[0]
    foc_Comb.name = para[0]
    return rtn_foc,vol_foc,foc_Comb

In [8]:
fre = 240
get_data_para_lst = [(name,fre) for name in names]
with Pool(16) as p:
    get_data_res_lst = list(tqdm(p.imap(get_foc, get_data_para_lst), total=len(get_data_para_lst)))  
foc_rtn_list,foc_vol_list,foc_Comb_list = zip(*get_data_res_lst)
foc_rtn = pd.concat(foc_rtn_list,axis=1).sort_index()
foc_vol = pd.concat(foc_vol_list,axis=1).sort_index()
foc_Comb = pd.concat(foc_Comb_list,axis=1).sort_index()
foc_rtn = foc_rtn.rolling(window = 15,min_periods = 1).mean()
foc_vol = foc_vol.rolling(window = 15,min_periods = 1).mean()
foc_Comb = foc_Comb.rolling(window = 15,min_periods = 1).mean()

100%|████████████████████████████████████████████████████████████████| 34/34 [00:08<00:00,  4.06it/s]


## DW统计量
$D W=\frac{\sum_{i=2}^n\left(e_i-e_{i-1}\right)^2}{\sum_{i=1}^n\left(e_i-\bar{e}\right)^2}$

In [9]:
def dw(data):
    result = (data.diff()**2).sum()/((data-data.mean())**2).sum()
    return result

In [10]:
def get_dw(para):
    data = get_data(para)
    rtn_dw = data.ret.resample('D').apply(lambda x: dw(x))
    vol_dw = data.v_fra.resample('D').apply(lambda x: dw(x))
    dw_Comb = (rtn_dw+vol_dw)*0.5
    rtn_dw.name = para[0]
    vol_dw.name = para[0]
    dw_Comb.name = para[0]
    return rtn_dw,vol_dw,dw_Comb

In [11]:
fre = 240
get_data_para_lst = [(name,fre) for name in names]
with Pool(16) as p:
    get_data_res_lst = list(tqdm(p.imap(get_dw, get_data_para_lst), total=len(get_data_para_lst)))  
DW_rtn_list,DW_vol_list,DW_Comb_list = zip(*get_data_res_lst)
DW_rtn = pd.concat(DW_rtn_list,axis=1).sort_index()
DW_vol = pd.concat(DW_vol_list,axis=1).sort_index()
DW_Comb = pd.concat(DW_Comb_list,axis=1).sort_index()
DW_rtn = DW_rtn.rolling(window = 15,min_periods = 1).mean()
DW_vol = DW_vol.rolling(window = 15,min_periods = 1).mean()
DW_Comb = DW_Comb.rolling(window = 15,min_periods = 1).mean()

100%|████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.58it/s]


## 残差自相关系数

In [12]:
def rho(data):
    data_s1 = np.array([np.nan] + list(data[:-1]))
    valid_indices = np.logical_and(~np.isnan(data), ~np.isnan(data_s1))
    data_valid = data[valid_indices]
    data_s1_valid = data_s1[valid_indices]
    data_res = np.nan
    # 防止所有位置均为空值的情况
    if not np.all(~valid_indices):
        data_s1_c = sm.add_constant(data_s1_valid)
        data_model = sm.OLS(data_valid, data_s1_c)
        data_results = data_model.fit()
        try:
            data_res = data_results.params[1]
        except:
            pass  
    return data_res

In [13]:
fre = 240
get_data_para_lst = [(name,fre) for name in names]
with Pool(16) as p:
    get_data_res_lst = list(tqdm(p.imap(get_data, get_data_para_lst), total=len(get_data_para_lst)))  
v_fra = pd.DataFrame([res['v_fra'] for res in get_data_res_lst], index=names).T.sort_index()
ret = pd.DataFrame([res['ret'] for res in get_data_res_lst], index=names).T.sort_index()
# .values按照行取（我希望其按照列取）
fre_num = int(24*60/fre)
fre_len = int(v_fra.shape[0]*v_fra.shape[1]/fre_num)
v_fra_list = np.reshape(v_fra.T.values, (fre_len,fre_num))
ret_list = np.reshape(ret.T.values, (fre_len,fre_num))

with Pool(16) as p:
    rho_res_v_fra_lst = list(tqdm(p.imap(rho, v_fra_list), total=len(v_fra_list))) 
    rho_res_ret_lst = list(tqdm(p.imap(rho, ret_list), total=len(ret_list))) 

day_num = len(np.unique(v_fra.index.date))
name_num = len(names)
rho_res_v_fra_lst_reshape = np.reshape(rho_res_v_fra_lst, (name_num,day_num))
rho_res_ret_lst_reshape = np.reshape(rho_res_ret_lst, (name_num,day_num))
rtn_rho = pd.DataFrame(rho_res_ret_lst_reshape,index = names,columns = np.unique(v_fra.index.date)).T
vol_rho = pd.DataFrame(rho_res_v_fra_lst_reshape,index = names,columns = np.unique(v_fra.index.date)).T
rho_Comb = (rtn_rho+vol_rho)*0.5

rtn_rho = rtn_rho.rolling(window = 15,min_periods = 1).std()
vol_rho = vol_rho.rolling(window = 15,min_periods = 1).std()
rho_Comb = rho_Comb.rolling(window = 15,min_periods = 1).std()

100%|███████████████████████████████████████████████████████| 63682/63682 [00:02<00:00, 21940.45it/s]


# 个股的非同步交易性与高阶自相关性

$L B=n(n+2) \sum_{k=1}^m\left(\frac{\hat{p}_k^2}{n-k}\right)$

In [147]:
'''
参数设定：
Q:统计量计算：最高到n=5阶自相关性
回望区间从3*n(15)到6*n(30)
'''

def get_Q(data):
    h = 5
    Q = np.nan
    if len(data) >= h:
        acf = sm.tsa.acf(data, nlags=h)
        n = len(data)
        Q = n * (n + 2) * sum(acf[k]**2 / (n - k) for k in range(1, h + 1))
    return Q

In [148]:
fre = 5
get_data_para_lst = [(name,fre) for name in names]
with Pool(34) as p:
    get_data_res_lst = list(tqdm(p.imap(get_data, get_data_para_lst), total=len(get_data_para_lst)))  
v_fra_5 = pd.DataFrame([res['v_fra'] for res in get_data_res_lst], index=names).T.sort_index()
ret_5 = pd.DataFrame([res['ret'] for res in get_data_res_lst], index=names).T.sort_index()
fre_len= int(24*60/fre)
fre_num = int(ret_5.shape[0]*ret_5.shape[1]/fre_len)
v_fra_5_list = np.reshape(v_fra_5.T.values,(fre_num,fre_len))
ret_5_list = np.reshape(ret_5.T.values,(fre_num,fre_len))
v_fra_5_lst = []
ret_5_lst = []
for i in range(24):
    v_fra_5_lst+=list(v_fra_5_list[:,-(12+6*i):])
    ret_5_lst+=list(ret_5_list[:,-(12+6*i):])

with Pool(32) as p:
    v_fra_5_Q_period_lst = list(tqdm(p.imap(get_Q, v_fra_5_lst), total=len(v_fra_5_lst))) 
    ret_5_Q_period_lst = list(tqdm(p.imap(get_Q, ret_5_lst), total=len(ret_5_lst))) 
fre_len1= int(len(v_fra_5_Q_period_lst)/24)
fre_num1 = int(24)
rtn_LBQ_all_data= np.reshape(ret_5_Q_period_lst,(fre_num1,fre_len1)).T
vol_LBQ_all_data= np.reshape(v_fra_5_Q_period_lst,(fre_num1,fre_len1)).T
rtn_LBQ_std_data = np.std(rtn_LBQ_all_data, axis=1)
vol_LBQ_std_data = np.std(vol_LBQ_all_data, axis=1)
fre_len2= int(len(rtn_LBQ_std_data)/len(names))
fre_num2 = int(len(names))
rtn_LBQ_lst = np.reshape(rtn_LBQ_std_data,(fre_num2,fre_len2))
vol_LBQ_lst = np.reshape(vol_LBQ_std_data,(fre_num2,fre_len2))
rtn_LBQ = pd.DataFrame(rtn_LBQ_lst,index = names ,columns = np.unique(ret_5.index.date)).T
vol_LBQ = pd.DataFrame(vol_LBQ_lst,index = names ,columns = np.unique(ret_5.index.date)).T
LBQ_Comb = (rtn_LBQ+vol_LBQ)*0.5
LBQ_Comb = LBQ_Comb.rolling(window = 15,min_periods = 1).mean()

100%|███████████████████████████████████████████████████| 1528368/1528368 [00:59<00:00, 25803.83it/s]


# 高波动率下的收益率均值

In [114]:
def get_highStdRtn_mean(data):
    data_std = np.array([np.std(data[max(i - 6 + 1, 0):i+1]) for i in range(len(data))])
    percentile_80 = np.percentile(data_std, 80)
    data_above_80_percentile = data[data_std > percentile_80]
    return np.mean(data_above_80_percentile)

In [116]:
'''
考虑到虚拟货币的交易并不是每天间断的
'''
ret_list = np.reshape(ret_5.T.values,(fre_num,fre_len))

with Pool(24) as p:
    highStdRtn_mean_lst = list(tqdm(p.imap(get_highStdRtn_mean, ret_list), total=len(ret_list))) 

day_num = len(np.unique(v_fra.index.date))
name_num = len(names)
highStdRtn_mean_lst_reshape = np.reshape(highStdRtn_mean_lst, (name_num,day_num))

highStdRtn_mean = pd.DataFrame(highStdRtn_mean_lst_reshape,columns = np.unique(ret_5.index.date),index = names).T


100%|████████████████████████████████████████████████████████| 63682/63682 [00:13<00:00, 4846.74it/s]


# 考虑股价自相关性的风险度量因子

VaR:投資組合在一定期間內可能會受到的最大損失有多少。           
VaR计算原理：   
方差-協方差法 (Parametric VaR) : 基於收益率的正態分布假設, 利用投資組合收益率的均值和標準差來計算 VaR。

公式如下：
$\mathrm{VaR}=\mu-(\mathrm{z} * \sigma)$        
$\mu$ : 投資組合報酬率的平均值。        
$\mathrm{z}$ : 在信心水準相對應的正態分布分位數。         
$\sigma$ : 投資組合報酬率的標準差。        

我们现在需要得出股价的分布：            
具体来说, 我们首先假设股票价格服从以下分布
$$
\ln P_{t+n}-\ln P_t \sim N\left[\left(\mu-\frac{\sigma^2}{2}\right) \cdot n, \sigma \cdot \sqrt{n}\right]
$$

其中, $P_{t+n}$ 为末来时刻的股票价格, $P_t$ 为当前时刻的股票价格。我们设置 $n=1$,即价格时差为 1 分钟。由于需要考虑到分钟级股价存在自相关性, 我们进一步假设其 $\ln P_{t+1}$ 与 $\ln P_t$ 之间的相关系数为 $\rho$.

在此基础之上, 我们假设 $\left(\ln P_{t+1}, \ln P_t\right)$ 服从二元正态分布, 且两者具有一样的均值和标准差。由此, 我们可得
$$
\left(\ln P_{t+1}, \ln P_t\right) \sim N\left(\mu, \mu, \sigma^2, \sigma^2, \rho\right)
$$

VaR的计算公式为:
$$
\mathrm{VaR}=-P_t \times\left(Z_\alpha \times \sigma+\left(\mu-\frac{\sigma^2}{2}\right)\right)
$$

其中，
- $P_t$ 是当日收盘价;
- $Z_\alpha$ 是对应于置信水平 $\alpha$ 的标准正态分位数；
- $\sigma$ 是分钟级对数价格的标准差;
- $\mu$ 是期望收益率。

这里的 $\mu$ 和 $\sigma$ 均对应着股票价格的对数收益率的期望值和标准差。望收益率。

$\mathbf{y}_1 \mid \mathbf{y}_2 \sim N_r\left(\boldsymbol{\mu}_1+\boldsymbol{\Sigma}_{12} \boldsymbol{\Sigma}_{22}^{-1}\left(\mathbf{y}_2-\boldsymbol{\mu}_2\right), \boldsymbol{\Sigma}_{11}-\boldsymbol{\Sigma}_{12} \boldsymbol{\Sigma}_{22}^{-1} \boldsymbol{\Sigma}_{21}\right)$

In [117]:
def get_VaR(data):
    mu = np.mean(data)
    sigma = np.std(data)
    data1 = np.array([np.nan] + list(data[:-1]))
    valid_indices = np.logical_and(~np.isnan(data), ~np.isnan(data1))
    rho = np.corrcoef(data[valid_indices], data1[valid_indices])[0][1]
    Sigma = rho*np.array([[1,rho],[rho,1]])
    C = np.array([[1,-1],[0,1]])
    nu_Sigma = np.dot(np.dot(C.T,Sigma),C)
    ad_mu = nu_Sigma[0][1]/nu_Sigma[1][1]*(data[-1]-mu)
    ad_sigma = nu_Sigma[0][0] - nu_Sigma[0][1]/nu_Sigma[1][1]*nu_Sigma[1][0]
    VaR = ad_mu - 1.96*ad_sigma**0.5
    return VaR

In [118]:
fre = 1
get_data_para_lst = [(name,fre) for name in names]
with Pool(34) as p:
    get_data_res_lst = list(tqdm(p.imap(get_data_simple2, get_data_para_lst), total=len(get_data_para_lst)))  
close_1 = pd.concat(get_data_res_lst,axis = 1).sort_index()
fre_num = int(24*60/fre)
fre_len = int(close_1.shape[0]*close_1.shape[1]/fre_num)
close_list = np.reshape(close_1.T.values, (fre_len,fre_num))

with Pool(24) as p:
    VaR_lst = list(tqdm(p.imap(get_VaR, close_list), total=len(close_list))) 

day_num = len(np.unique(close_1.index.date))
name_num = len(names)
VaR_lst_reshape = np.reshape(VaR_lst, (name_num,day_num))

VaR = pd.DataFrame(VaR_lst_reshape,columns = np.unique(close_1.index.date),index = names).T
VaR = VaR.rolling(window = 15,min_periods = 1).std()

100%|███████████████████████████████████████████████████████| 63682/63682 [00:06<00:00, 10385.91it/s]


# “闪电崩盘”、连续下跌和“崩盘概率”因子

$\hat{\lambda} = \bar{x}$

In [119]:
def get_lambda(data):
    pos_lam = np.nan
    neg_lam = np.nan
    pos_lst = []
    neg_lst = []
    m = 0
    n = 0
    j = 0
    for i in data:
        if i <0 and j <0:
            n += 1
            if m != 0:
                neg_lst.append(m)
                m = 0
        elif i >0 and j >0:
            m += 1
            if n != 0:
                pos_lst.append(n)
                n = 0
        elif m != 0:
            neg_lst.append(m)
            m = 0
        elif n != 0:
            pos_lst.append(n)
            n = 0
        j = i
    if len(pos_lst) > 0:
        pos_lam = np.mean(pos_lst)
    if len(neg_lst) > 0:
        neg_lam = np.mean(neg_lst)    
    return pos_lam,neg_lam

In [120]:
def get_prob(data):
    poisson_dist = poisson(mu=data[0]*10)
    prob = poisson_dist.cdf(data[2]*10)-poisson_dist.cdf(data[1]*10)
    return prob

In [121]:
fre = 1
get_data_para_lst = [(name,fre) for name in names]
with Pool(34) as p:
    get_data_res_lst = list(tqdm(p.imap(get_data_simple, get_data_para_lst), total=len(get_data_para_lst)))  
ret_1 = pd.concat(get_data_res_lst,axis = 1).sort_index()

100%|████████████████████████████████████████████████████████████████| 34/34 [00:03<00:00, 10.62it/s]


In [122]:
fre_num = int(24*60)
fre_len = int(ret_1.shape[0]*ret_1.shape[1]/fre_num)
ret_list = np.reshape(ret_1.T.values, (fre_len,fre_num))

with Pool(16) as p:
    lambda_lst = list(tqdm(p.imap(get_lambda, ret_list), total=len(ret_list))) 
pos_lambda_lst ,neg_lambda_lst = zip(*lambda_lst)
pos_lambda_lst_reshape = np.reshape(pos_lambda_lst, (name_num,day_num))
neg_lambda_lst_reshape = np.reshape(neg_lambda_lst, (name_num,day_num))
pos_lambda = pd.DataFrame(pos_lambda_lst_reshape,columns = np.unique(ret_1.index.date) ,index = names).T
neg_lambda = pd.DataFrame(neg_lambda_lst_reshape,columns = np.unique(ret_1.index.date) ,index = names).T

100%|███████████████████████████████████████████████████████| 63682/63682 [00:05<00:00, 11378.77it/s]


In [123]:
ne_lambda_lst = neg_lambda.T.values.flatten().tolist()
x_k = neg_lambda.quantile(0.75,axis = 1).values.tolist()*len(names)
x = pos_lambda.quantile(0.5,axis = 1).values.tolist()*len(names)
data_lst = ne_lambda_lst+x+x_k
fre_num = int(3)
fre_len = int(len(data_lst)/fre_num)
data_lst_reshape = np.reshape(data_lst,(fre_num,fre_len)).T
with Pool(24) as p:
    prob_list = list(tqdm(p.imap(get_prob, data_lst_reshape), total=len(data_lst_reshape))) 

fre_num = int(len(names))
fre_len = int(len(prob_list)/fre_num)
prob_list_reshape = np.reshape(prob_list,(fre_num,fre_len))
    
prob_df = pd.DataFrame(prob_list_reshape,index = names,columns = np.unique(ret_1.index.date)).T
flashCrashProb = prob_df.rolling(window =15,min_periods=1).std()

100%|███████████████████████████████████████████████████████| 63682/63682 [00:03<00:00, 20835.40it/s]


# 因子合成


In [124]:
# 截面标准化
def scale(data):
    scaled_data = (data-np.mean(data))/np.std(data)
    return scaled_data

In [151]:
para_lst = foc_Comb.values.tolist() + DW_Comb.values.tolist() + rho_Comb.values.tolist() + LBQ_Comb.values.tolist() + highStdRtn_mean.values.tolist() + VaR.values.tolist() + flashCrashProb.values.tolist() 
with Pool(24) as p:
    final_res_lst = list(tqdm(p.imap(scale,para_lst),total = len(para_lst)))
_num = int(7)
_len = int(len(para_lst)*len(names)/7)
final_res_lst_reshape = np.reshape(final_res_lst,(_num,_len))
foc_Comb_scale_lst = final_res_lst_reshape[0] #升序
DW_Comb_scale_lst = final_res_lst_reshape[1] #降序
rho_Comb_scale_lst = final_res_lst_reshape[2] #升序
LBQ_Comb_scale_lst = final_res_lst_reshape[3] #降序
highStdRtn_mean_scale_lst = final_res_lst_reshape[4] #升序
VaR_scale_lst = final_res_lst_reshape[5] #降序
flashCrashProb_scale_lst = final_res_lst_reshape[6] #降序
fre_num = int(len(foc_Comb_scale_lst)/len(names))
fre_len = int(len(names))
foc_Comb_scale_lst_reshape = np.reshape(foc_Comb_scale_lst,(fre_num,fre_len))
foc_Comb_scale = pd.DataFrame(foc_Comb_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
DW_Comb_scale_lst_reshape = np.reshape(DW_Comb_scale_lst,(fre_num,fre_len))
DW_Comb_scale = pd.DataFrame(DW_Comb_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
rho_Comb_scale_lst_reshape = np.reshape(rho_Comb_scale_lst,(fre_num,fre_len))
rho_Comb_scale = pd.DataFrame(rho_Comb_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
LBQ_Comb_scale_lst_reshape = np.reshape(LBQ_Comb_scale_lst,(fre_num,fre_len))
LBQ_Comb_scale = pd.DataFrame(LBQ_Comb_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
highStdRtn_mean_scale_lst_reshape = np.reshape(highStdRtn_mean_scale_lst,(fre_num,fre_len))
highStdRtn_mean_scale = pd.DataFrame(highStdRtn_mean_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
VaR_scale_lst_reshape = np.reshape(VaR_scale_lst,(fre_num,fre_len))
VaR_scale = pd.DataFrame(VaR_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
flashCrashProb_scale_lst_reshape = np.reshape(flashCrashProb_scale_lst,(fre_num,fre_len))
flashCrashProb_scale = pd.DataFrame(flashCrashProb_scale_lst_reshape,index = foc_Comb.index,columns = foc_Comb.columns)
final_factor = (foc_Comb_scale - DW_Comb_scale + rho_Comb_scale - LBQ_Comb_scale + highStdRtn_mean_scale - VaR_scale - flashCrashProb_scale)/7

100%|███████████████████████████████████████████████████████| 13111/13111 [00:00<00:00, 26137.49it/s]


In [153]:
final_factor.dropna()

,BONK,ETH,ICP,INJ,SEI,RUNE,1000SATS,LINK,XRP,APE,...,UNI,DOT,EOS,BAKE,LTC,BCH,SOL,ATOM,ARPA,XLM
0,,,,,,,,,,,,,,,,,,,,,
2023-12-17,-0.096721,0.246700,0.429392,-0.053561,-0.448566,0.065520,-0.679078,-0.073822,-0.191071,0.093019,...,-0.035663,-0.009706,0.456071,0.375371,-0.015596,-0.412610,-0.063745,0.127785,0.305625,0.235037
2023-12-18,0.343446,0.100359,0.421572,0.059880,-0.537795,0.061992,-1.058607,0.119424,-0.144305,0.139270,...,-0.073155,0.114948,0.522755,0.526505,0.017648,-0.345046,-0.092791,0.398946,0.260992,0.277132
2023-12-19,0.387786,0.162756,0.181400,-0.047805,-0.493545,-0.061912,0.064149,0.138422,0.010528,-0.111921,...,-0.245571,-0.116624,0.512589,0.543805,0.060856,-0.302129,-0.154850,0.075782,0.174548,0.210331
2023-12-20,0.773807,-0.063922,0.315020,0.010023,-0.127084,0.087589,-0.904341,-0.039145,0.211293,-0.115416,...,-0.137858,0.187447,0.650537,0.092372,-0.081075,-0.090314,-0.020598,0.349297,0.209923,0.308984
2023-12-21,0.289180,-0.193021,0.054124,-0.361369,-0.645626,-0.359137,-0.530477,0.038167,0.260398,-0.373805,...,-0.227811,0.531402,0.720718,0.163050,0.150311,0.016812,0.394554,0.195678,0.279732,0.261482
2023-12-22,0.316141,-0.057004,0.184514,-0.088287,-0.427079,-0.473246,-0.606558,-0.081585,0.295362,-0.284992,...,-0.230579,0.599085,0.886723,0.248401,0.094570,-0.069239,0.020546,0.381171,0.159817,0.373224
2023-12-23,0.086853,-0.075861,0.324379,-0.104366,-0.320116,-0.539802,-0.356571,0.255027,0.446718,-0.173730,...,-0.013401,0.405978,0.724033,-0.418580,0.299174,-0.092621,0.064519,0.634787,0.121100,0.356106
2023-12-24,0.457794,-0.114853,-0.066994,0.277719,-0.404603,-0.319931,-0.584709,-0.077396,0.415117,-0.199956,...,-0.254217,0.273112,0.673173,0.333584,0.252125,-0.004272,0.253761,0.278976,0.149067,0.383672
2023-12-25,0.356762,-0.116742,-0.069643,0.234336,-0.151151,-0.589628,0.311011,-0.197212,0.285593,-0.090304,...,-0.164108,0.282523,0.675697,-0.258984,0.311995,0.079946,0.321354,0.278411,0.094669,0.336830
